## Function calling

This new feature proves beneficial in a wide array of situations. It can assist in designing chatbots capable of interacting with various APIs, facilitate task automation, and enable extraction of organized information from inputs expressed in natural language.

In [134]:
!pip install --upgrade langchain

Ensure you got the AT LEAST Version 0.0.200

In [135]:
import pkg_resources


def print_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f"The version of the {package_name} library is {version}.")
    except pkg_resources.DistributionNotFound:
        print(f"The {package_name} library is not installed.")


print_version("langchain")

The version of the langchain library is 0.0.201.


In [136]:
from dotenv import load_dotenv
import os
import openai
import json

load_dotenv()
API_KEY = os.environ.get("API_KEY")

First, you need to define a function. This one mimics a database query with a "fake" price.

In [137]:
def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [138]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [139]:
def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions,
    )
    message = response["choices"][0]["message"]
    return message

In [140]:
chat("What is the capital of france?")

<OpenAIObject at 0x1df198b3fb0> JSON: {
  "content": "The capital of France is Paris.",
  "role": "assistant"
}

In [141]:
query = "How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x1df198b33b0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"pizza_name\": \"Salami\"\n}",
    "name": "get_pizza_info"
  },
  "role": "assistant"
}

In [142]:
if message.get("function_call"):
    function_name = message["function_call"]["name"]
    function_response = get_pizza_info(
        pizza_name=message.get("pizza_name"),
    )

    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": query},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

second_response

InvalidRequestError: {'name': None, 'price': '10.99'} is not of type 'string' - 'messages.2.content'

### LangChain 

LangChain allows you to use functions too, but currently (15.06.2023) it is kind of hacky to do it. 


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
message = llm.predict_messages(
    [HumanMessage(content="What is the capital of france?")], functions=functions
)
message

In [ ]:
query = "How much does Pizza Salami cost in the restaurant?"
message_pizza = llm.predict_messages([HumanMessage(content=query)], functions=functions)
message_pizza

In [ ]:
message.additional_kwargs

In [ ]:
str(message_pizza.additional_kwargs)

In [ ]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(message_pizza.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": message_pizza.additional_kwargs["function_call"]["name"]
            },
            content=query,
        ),
    ],
    functions=functions,
)
second_response

### Use Tools

You can convert Tools to functions, both the Tools provided by langchain and also your own tools. The Tool you see is there to show you the interface of a tool, it actually does nothing really do anything useful ;-) 

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class StupidJokeTool(BaseTool):
    name = "StupidJokeTool"
    description = "Tool to explain stupid jokes"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return f"LOL: {query}"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("joke tool does not support async")

In [ ]:
from langchain.tools import format_tool_to_openai_function, MoveFileTool


tools = [StupidJokeTool(), MoveFileTool()]
functions = [format_tool_to_openai_function(t) for t in tools]
functions

In [ ]:
query = "Why does the chicken cross the road? To get to the other side"
output = llm.predict_messages([HumanMessage(content=query)], functions=functions)
output

In [ ]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(output.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": output.additional_kwargs["function_call"]["name"]
            },
            content="""
                {query}
            """,
        ),
    ],
    functions=functions,
)
second_response.content